In the case when the ground state is a common state of both spin species, we resort to a spin-1 formulation, the following code implements this description

In [ ]:
using DifferentialEquations, PyPlot, Statistics, LaTeXStrings, PyCall, LinearAlgebra

In [ ]:
ENV["PYTHON"] = "C:\\Users\\admin\\Anaconda3\\envs\\data_vis\\python.exe"

LogNorm = pyimport("matplotlib.colors").LogNorm
cmap = pyimport("cmap")

lajolla = cmap.Colormap("crameri:lajolla").to_mpl();
balance = cmap.Colormap("cmocean:balance").to_mpl();
oslo = cmap.Colormap("crameri:oslo").to_mpl();
lipari = cmap.Colormap("crameri:lipari").to_mpl();
eclipse = cmap.Colormap("cmasher:eclipse").to_mpl();

# Configure LaTeX rendering and fonts
PyPlot.matplotlib[:rc]("text", usetex=true)
PyPlot.matplotlib[:rc]("font", family="serif", serif=["mathpazo"], size=18)  # Base font size
PyPlot.matplotlib[:rc]("axes", titlesize=22)             # Axis title
PyPlot.matplotlib[:rc]("axes", labelsize=20)             # Axis labels
PyPlot.matplotlib[:rc]("xtick", labelsize=18)            # X-ticks
PyPlot.matplotlib[:rc]("ytick", labelsize=18)            # Y-ticks
PyPlot.matplotlib[:rc]("legend", fontsize=18)            # Legend
PyPlot.matplotlib[:rc]("figure", titlesize=24)           # Figure title
PyPlot.svg(true)
# LaTeX preamble packages
PyPlot.matplotlib[:rc]("text.latex", preamble="\\usepackage{amsmath}\\usepackage{amsfonts}\\usepackage{amssymb}\\usepackage{lmodern}")

In [ ]:
# Define the system of ODEs
function coupled_eqs!(du, u, p, t)
    Δ1, Δ2, g, φ, Δ, κ, η = p
    λ1, λ2, λ3, λ4, λ5, λ6, λ7, λ8, α_re, α_im = u
    
    α = α_re + im * α_im
    αc = conj(α)
    expp = cos(φ) + im * sin(φ)
    expm = conj(expp)
    
    du[1] = Δ1 * λ2 + im* g * ( λ3*(α - αc) + expm*αc/2 *(-im * λ5 + λ4) + expp*α/2 * (-im * λ5 - λ4) )  # ∂ₜ λ1
    du[2] = -Δ1 * λ1 + im* g * ( im* λ3*(α + αc) + expm*αc/2 *(im * λ4 + λ5) + expp*α/2 * (im * λ4 - λ5) )    # ∂ₜ λ2
    du[3] = im* g * ( -im* λ2*(α + αc) + λ1 * (αc - α) + expm*αc/2 *(im * λ7 - λ6) + expp*α/2 * (im * λ7 - λ6) )   # ∂ₜ λ3
    
    du[4] = (Δ1  - Δ2/2) * λ5 + im* g * ( αc/2 * (im*λ7 + λ6) + α/2 * (im*λ7 - λ6) + expm*αc/2 *(-im * λ2 - λ1) + expp*α/2 * (-im * λ2 + λ1) )   # ∂ₜ λ4
    du[5] = -(Δ1  - Δ2/2) * λ4 + im* g * ( αc/2 * (-im*λ6 + λ7) + α/2 * (-im*λ6 - λ7) + expm*αc/2 *(im * λ1 - λ2) + expp*α/2 * (-im * λ1 + λ2) )  # ∂ₜ λ5

    du[6] = -Δ2/2 * λ7 + im * g * ( αc/2 * (im*λ5 - λ4) + α/2 * (im*λ5 + λ4) + 1/2 * ( expp*α - expm*αc)*(sqrt(3)*λ8 - λ3 ) )  # ∂ₜ λ6 (VERIFY)
    du[7] = Δ2/2 * λ6 + im * g * ( αc/2 * (-im*λ4 - λ5) + α/2 * (-im*λ4 + λ5) + 1/2 * ( expp*α + expm*αc)*(sqrt(3)*λ8 - λ3 ) ) # ∂ₜ λ7 (VERIFY)

    du[8] = -im*sqrt(3)/2 * g * (αc*expm * (im*λ7-λ6) +α*expp * (im*λ7 + λ6)) #∂t λ8
    
    dα = (Δ * im - κ / 2) * α - im*g * ( ( λ1 - im*λ2) + expm*(λ6 - im*λ7) ) - η # ∂t α
    du[7] = real(dα)  # Re(α_dot)
    du[8] = imag(dα)  # Im(α_dot)
    end

In [ ]:
# Fixed parameters
Δ1, Δ2 = 0, 0
g, κ = 0.1, 1.0
Δ = 0.0
ϕ = 0
η = g
# Time steps
tend = 100;
t_list = range(0.0, tend, length=1001);

# Initial conditions
u0 = [1, 0.0, 0, 0, 0.0, 1, 0, 0, 0, 0];


In [ ]:
params = (Δ1, Δ2, g, ϕ, Δ, κ, η)
prob = ODEProblem(coupled_eqs!, u0, (0.0, tend), params)
sol = solve(prob, Tsit5(), saveat=t_list, reltol=1e-6, abstol=1e-9, maxiters=1e10);

In [ ]:
# Obtain populations of each level in the spin-1 system
N0 =  1/3 .- sol[3,:] + 1/sqrt(3) .* sol[8,:];
N1 = 1/3 .+ sol[3,:] + 1/sqrt(3) .* sol[8,:];
N2 = 1/3 .- 1/sqrt(3) .* sol[8,:];

In [ ]:
Nt = N0 + N1 + N2

In [ ]:
# Plot populations

fig, ax = subplots(1, 1, figsize=(8, 6))
ax.plot(sol.t, N0, color=lajolla(0.1), label=L"$N_0$", lw=2)
ax.plot(sol.t, N1, color=lajolla(0.5), label=L"$N_1$", lw=2)
ax.plot(sol.t, N2, color=lajolla(0.9), label=L"$N_2$", lw=2)
ax.set_xlabel(L"$t$", fontsize=20)
ax.set_ylabel(L"$N_i$", fontsize=20)
fig.tight_layout()
display(fig)

In [ ]:
adaga = sol[9,:].^2 .+ sol[10,:].^2 

fig, ax = subplots(1, 1, figsize=(8, 6))
ax.plot(sol.t, adaga, lw=2)
ax.set_xlabel(L"$t$", fontsize=20)
ax.set_ylabel(L"$n$", fontsize=20)
fig.tight_layout()
display(fig)